In [20]:
import numpy as np
from sklearn.naive_bayes import BernoulliNB
import time
import gzip
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression
from dataset import SantanderDataset
from average_precision import mapk
from genetic_search import genetic_search
import pandas as pd
dataset_root = '../'
dataset = SantanderDataset(dataset_root)

It took 3 seconds to load the dataset
It took 4 seconds to load the dataset
It took 6 seconds to load the dataset
It took 1 seconds to load the dataset
(1375686, 1375686)
(561234, 561234)


In [16]:
#Try training
start_time = time.time()
msg = {'train_month': 5,
       'eval_month': 5,
      'input_columns': [],
      'use_product': True,
      'use_change': False}
msg_copy = msg.copy()
msg_copy['train'] = False
if not 'month' in msg_copy.keys():
    msg_copy['month'] = msg_copy['eval_month']
ret = dataset.get_data(msg_copy)

In [21]:
limit_rows   = 20000000
dictionary_types = {
                            "sexo":'category',
                            "ult_fec_cli_1t":str,
                            "indresi":'category',
                            "indext":'category',
                            "indrel":'category',
                            "indfall":'category',
                            "nomprov":'category',
                            "segmento":'category',
                            "ind_empleado":'category',
                            "pais_residencia":'category',
                            "antiguedad":np.int16,
                            "ind_nuevo":'category',
                            'indrel_1mes':'category',
                            'tiprel_1mes':'category',
                            'canal_entrada':'category',
                            "age":np.int8,
                            "ind_actividad_cliente":'category',
                            "ind_ahor_fin_ult1":np.int8,
                            "ind_aval_fin_ult1":np.int8,
                            "ind_cco_fin_ult1":np.int8,
                            "ind_cder_fin_ult1":np.int8,
                            "ind_cno_fin_ult1":np.int8,
                            "ind_ctju_fin_ult1":np.int8,
                            "ind_ctma_fin_ult1":np.int8,
                            "ind_ctop_fin_ult1":np.int8,
                            "ind_ctpp_fin_ult1":np.int8,
                            "ind_deco_fin_ult1":np.int8,
                            "ind_deme_fin_ult1":np.int8,
                            "ind_dela_fin_ult1":np.int8,
                            "ind_ecue_fin_ult1":np.int8,
                            "ind_fond_fin_ult1":np.int8,
                            "ind_hip_fin_ult1":np.int8,
                            "ind_plan_fin_ult1":np.int8,
                            "ind_pres_fin_ult1":np.int8,
                            "ind_reca_fin_ult1":np.int8,
                            "ind_tjcr_fin_ult1":np.int8,
                            "ind_valo_fin_ult1":np.int8,
                            "ind_viv_fin_ult1":np.int8,
                            "ind_nomina_ult1":np.int8,
                            "ind_nom_pens_ult1":np.int8,
                            "ind_recibo_ult1":np.int8,
                            "ind_ahor_fin_ult1_change":'category',
                            "ind_aval_fin_ult1_change":'category',
                            "ind_cco_fin_ult1_change":'category',
                            "ind_cder_fin_ult1_change":'category',
                            "ind_cno_fin_ult1_change":'category',
                            "ind_ctju_fin_ult1_change":'category',
                            "ind_ctma_fin_ult1_change":'category',
                            "ind_ctop_fin_ult1_change":'category',
                            "ind_ctpp_fin_ult1_change":'category',
                            "ind_deco_fin_ult1_change":'category',
                            "ind_deme_fin_ult1_change":'category',
                            "ind_dela_fin_ult1_change":'category',
                            "ind_ecue_fin_ult1_change":'category',
                            "ind_fond_fin_ult1_change":'category',
                            "ind_hip_fin_ult1_change":'category',
                            "ind_plan_fin_ult1_change":'category',
                            "ind_pres_fin_ult1_change":'category',
                            "ind_reca_fin_ult1_change":'category',
                            "ind_tjcr_fin_ult1_change":'category',
                            "ind_valo_fin_ult1_change":'category',
                            "ind_viv_fin_ult1_change":'category',
                            "ind_nomina_ult1_change":'category',
                            "ind_nom_pens_ult1_change":'category',
                            "ind_recibo_ult1_change":'category',
                            'product_buy':np.int8,
        }

eval_current = pd.read_csv(dataset_root + "input/eval_current_month_dataset.csv.gz",
                                   dtype=dictionary_types,
                                   nrows=limit_rows)

In [23]:
from feature_trans import LightGBMTrans

In [27]:
y = eval_current["fecha_dato"].values
X = eval_current.drop("fecha_dato", axis = 1).as_matrix

In [33]:
input_columns = ['ind_empleado']
n_values = [len(dataset.translation_dict[key].values())
                    for key in input_columns]

In [32]:
eval_current.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,tiprel_1mes,indresi,indext,canal_entrada,indfall,nomprov,ind_actividad_cliente,renta,segmento,new_products
0,1,15924,B,ES,V,52,1995-01-16,0,7348,1,...,A,S,N,KAT,N,"PALMAS, LAS",1,130903.68,02 - PARTICULARES,[18]
1,1,15925,F,ES,V,62,1995-01-16,0,7348,1,...,A,S,N,KAT,N,MADRID,1,42831.69,02 - PARTICULARES,[23]
2,1,15928,B,ES,H,49,1995-01-16,0,7348,1,...,A,S,N,KAT,N,MADRID,1,263023.98,01 - TOP,[18]
3,1,15965,F,ES,H,51,1995-01-16,0,7348,1,...,A,S,N,KAT,N,MADRID,1,44353.17,01 - TOP,"[7, 8]"
4,1,16026,F,ES,H,53,1995-02-06,0,7327,1,...,A,S,N,KAT,N,MADRID,1,151911.54,02 - PARTICULARES,[18]


In [34]:
n_values

[5]